In [ ]:
import getpass
import os
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
import json

with open(f'api.txt', errors='ignore') as f:
    api_key = f.read()
model = ChatCohere(cohere_api_key=api_key)

In [2]:
# Helper function to convert messages to JSON serializable format
def message_to_dict(message):
    if isinstance(message, HumanMessage):
        return {"type": "human", "content": message.content}
    elif isinstance(message, AIMessage):
        return {"type": "ai", "content": message.content}
    elif isinstance(message, SystemMessage):
        # Ignore SystemMessage objects
        return None
    else:
        raise ValueError("Unknown message type")

# Helper function to convert JSON data back to message objects
def dict_to_message(message_dict):
    if message_dict["type"] == "human":
        return HumanMessage(content=message_dict["content"])
    elif message_dict["type"] == "ai":
        return AIMessage(content=message_dict["content"])
    else:
        raise ValueError("Unknown message type")

# Save message history to JSON
def save_message_history(game: str, character: str, messages: list):
    json_file = f"{game}/characters/{character}/history_with_summary/message_history.json"
    
    # Convert message objects to dictionaries and filter out None values
    serializable_messages = [msg for msg in (message_to_dict(message) for message in messages) if msg is not None]
    
    with open(json_file, 'w') as f:
        json.dump({"messages": serializable_messages}, f, indent=4)

# Load message history from JSON
def load_message_history(game: str, character: str):
    json_file = f"{game}/characters/{character}/history_with_summary/message_history.json"
    
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
            # Convert dictionaries back to message objects
            return {"messages": [dict_to_message(msg) for msg in data["messages"]]}
    else:
        return {"messages": []}
    
def save_summary(game: str, character: str, summary: str):
    json_file = f"{game}/characters/{character}/history_with_summary/summary.json"
    
    print("Saving summary")
    
    # Save the summary to the separate JSON file as a string
    with open(json_file, 'w') as f:
        json.dump({"summary": [summary]}, f, indent=4)

def load_summary(game: str, character: str):
    json_file = f"{game}/characters/{character}/history_with_summary/summary.json"
    
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            data = json.load(f)
            return data.get("summary", "")  # Default to an empty string if "summary" not found
    else:
        return ""  # Return an empty string if the file does not exist
    


In [3]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are {character} from {game}.
            {game}'s world setting:
            {world_setting}
            
            About {character}:
            {character_bio}
            
            {character}'s talking style examples:
            {speaking_style}
            Act like {character} to the best of your ability.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import SystemMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END

from typing import Literal

first_call = True
memory = MemorySaver()

class State(MessagesState):
    summary: str
    character: str
    game: str

def call_model(state: State):
    global first_call
    character = state["character"]
    game = state["game"]
    
    with open(f'{game}\world_setting.txt', errors='ignore') as f:
        world_setting = f.read()
    
    with open(f'{game}\characters\{character}\character_bio.txt', errors='ignore') as f:
        character_bio = f.read()
    
    with open(f'{game}\characters\{character}\speaking_style.txt', errors='ignore') as f:
        speaking_style = f.read()
        
    if first_call:
        print(f"Current state: {state}")
        input_message = load_message_history(game, character)["messages"] + state["messages"]
        summary_message = load_summary(game, character)
        state["summary"] = summary_message
        first_call = False
    else:
        input_message = state["messages"]
    
    summary = state.get("summary", "")
    if summary:
        print("summary_found")
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + input_message
    else:
        messages = input_message    
    
    chain = prompt | model
    response = chain.invoke(
        {"messages": messages, "character": character, "game": game, "world_setting": world_setting, "character_bio": character_bio, "speaking_style": speaking_style}
    )
    
    updated_messages = messages + [response]
    save_message_history(game, character, updated_messages)
    
    messages_length = len(updated_messages)
    print(f"Messages length {messages_length}")
    
    if first_call:
        return {"messages": [response], "summary": summary_message}
    else:
        return {"messages": [response]}

def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END

def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    
    
    # Saving summary
    save_summary(state["game"], state["character"], response.content)
    
    print("Too many messages summarizing conversation.")
    return {"summary": response.content, "messages": delete_messages}

# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

In [ ]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])


character = "Varre"
game = "elden_ring"
config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="Who are you?")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:
input_message = HumanMessage(content="How many times have i said hello?")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:

input_message = HumanMessage(content="I dont think thats quite right")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:

input_message = HumanMessage(content="Why is it that you guide me?")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:
input_message = HumanMessage(content="Thanks...")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:
input_message = HumanMessage(content="Who are you?")
input_message.pretty_print()
for event in app.stream({"messages": input_message, "game": game, "character": character}, config, stream_mode="updates"):
    print_update(event)

In [ ]:
input_message = HumanMessage(content="Whats your name?")
output = app.invoke({"messages": input_message, "game": game, "character": character}, config)
output["messages"][-1].content

In [ ]:
values = app.get_state(config).values
values